In [202]:
# Necessary imports to work with netCDF 3D arrays
import xarray
import rioxarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import itertools

In [205]:
# Elevation map of the Americas
fn = 'test.nc'
pr = xarray.open_dataset(fn)

In [206]:
# pr is our data object containing the .nc contents
pr.dims

Frozen({'time': 1, 'reference_time': 1, 'feature_id': 2776738})

In [195]:
# Get a sense of how our data is structured
pr.coords['x']

<xarray.DataArray 'x' (x: 4608)>
array([-2303499.25, -2302499.25, -2301499.25, ...,  2301500.75,  2302500.75,
        2303500.75])
Coordinates:
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
    crs      int32 0
Attributes:
    long_name:            x coordinate of projection
    resolution:           1000
    standard_name:        projection_x_coordinate
    units:                m
    _CoordinateAxisType:  GeoX

In [209]:
# See the attributes of our data
print(pr)

<xarray.Dataset>
Dimensions:         (time: 1, reference_time: 1, feature_id: 2776738)
Coordinates:
  * time            (time) datetime64[ns] 2023-06-01T18:00:00
  * reference_time  (reference_time) datetime64[ns] 2023-06-01
  * feature_id      (feature_id) int32 101 179 181 ... 1180001803 1180001804
Data variables:
    crs             |S1 ...
    streamflow      (feature_id) float64 ...
    nudge           (feature_id) float64 ...
    velocity        (feature_id) float64 ...
    qSfcLatRunoff   (feature_id) float64 ...
    qBucket         (feature_id) float64 ...
    qBtmVertRunoff  (feature_id) float64 ...
Attributes: (12/19)
    TITLE:                      OUTPUT FROM NWM v2.2
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2023-06-01_00:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2023-06-01_18:00:00
    ...                         ...
    

In [208]:
pr.info()

xarray.Dataset {
dimensions:
	time = 1 ;
	reference_time = 1 ;
	feature_id = 2776738 ;

variables:
	datetime64[ns] time(time) ;
		time:long_name = valid output time ;
		time:standard_name = time ;
		time:valid_min = 28093020 ;
		time:valid_max = 28094040 ;
	datetime64[ns] reference_time(reference_time) ;
		reference_time:long_name = model initialization time ;
		reference_time:standard_name = forecast_reference_time ;
	|S1 crs() ;
		crs:transform_name = latitude longitude ;
		crs:grid_mapping_name = latitude longitude ;
		crs:esri_pe_string = GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision ;
		crs:spatial_ref = GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100

In [197]:
# we specify (time, lat, lon) - time is 0 because this is a static reading not a time series so we only need 

# pr[0, 100:105, 100:105] * pr.attrs['scale_factor']

In [198]:
lattitude = list(pr['x'].to_numpy())
longitude = list((pr['y']).to_numpy())
len(lattitude)
len(longitude)

3840

In [199]:
# Create a simple plot representation of our elevation map
# (pr[0, 0:500, 500:1500] * pr.attrs['scale_factor']).plot()

In [200]:
# (pr[0, 0:500, 500:1500] * pr.attrs['scale_factor']).plot()

In [8]:
# Here we are scraping all of the stream flow data from
# https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# This is all the streamflow data that USGS has available, when we call this the value we receive is discharge in cubic ft/s

def getStreamFlow():
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_streamflow = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(SiteTypeCode%20in%20(%27ST%27%2C%27ST-CA%27%2C%27ST-DCH%27%2C%27ST-TS%27))%20and%20(ParameterCode%20in%20(%2730208%27%2C%2730209%27%2C%2750042%27%2C%2750050%27%2C%2750051%27%2C%2772137%27%2C%2772138%27%2C%2772139%27%2C%2772177%27%2C%2772243%27%2C%2774072%27%2C%2781395%27%2C%2799060%27%2C%2799061%27%2C%2700056%27%2C%2700058%27%2C%2700059%27%2C%2700060%27%2C%2700061%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers,
    )
    
    return response_streamflow.json()['value']

In [9]:
# dicts containing all the information that pertains to our streamflow data from USGS map
# meaning that this is the network data that is called when https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# is first loaded

list_streamflow = getStreamFlow()

# Find all the unique statistic status, parameter codes, Flood Stage Status, Current Condition ID for streamflow data
statisticstatuscode = []
parametercode = []
floodstagestatus = []
currentconditionID = []

unique_statisticstatuscode = []
unique_parametercode = []
unique_floodstagestatus = []
unique_currentconditionID = []

# Loop to capture all elements in the list
for i in range(len(list_streamflow)):
    statisticstatuscode.append(list_streamflow[i]['StatisticStatusCode'])
    parametercode.append(list_streamflow[i]['ParameterCode'])
    floodstagestatus.append(list_streamflow[i]['FloodStageStatusCode'])
    currentconditionID.append(list_streamflow[i]['CurrentConditionID'])

# Finalize by finding the unique elements
unique_statisticstatuscode = list(set(statisticstatuscode))
unique_parametercode = list(set(parametercode))
unique_floodstagestatus = list(set(floodstagestatus))
unique_currentconditionID = list(set(currentconditionID))

In [10]:
# Key
# Value: Discharge in cubic feet/second - this value is more up-to-date in the scraping than the map as they have different update intervals
# RateOfChangeUnitPerHour: rate of change in ft/s per hour - also more up-to-date than the map
# SiteTypeCode: 

streamflow_df = pd.DataFrame.from_dict(getStreamFlow())

In [11]:
#Change the column names so that we can keep track of specific values
streamflow_df.rename(columns={'Value':'Discharge ft^3/s', 'RateOfChangeUnitPerHour':'RateofChange_Discharge'}, inplace=True)

In [12]:
streamflow_df

,AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Discharge ft^3/s,ValueFlagCode,RateofChange_Discharge,StatisticStatusCode,FloodStageStatusCode
0,USGS,01010000,"St. John River at Ninemile Bridge, Maine",ST,46.700556,-69.715556,65330,00060,2023-05-25T12:00:00Z,EDT,967.0,None,0.0,P0_10,NOFLOOD
1,USGS,01010070,"Big Black River near Depot Mtn, Maine",ST,46.893889,-69.751667,65336,00060,2023-05-25T12:00:00Z,EDT,112.0,None,2.0,P0_10,NOFLOOD
2,USGS,01010500,"St. John River at Dickey, Maine",ST,47.113056,-69.088056,65342,00060,2023-05-25T12:30:00Z,EDT,2160.0,None,0.0,P0_10,NOFLOOD
3,USGS,01011000,"Allagash River near Allagash, Maine",ST,47.069722,-69.079444,65349,00060,2023-05-25T12:00:00Z,EDT,1420.0,None,20.0,P0_10,NOFLOOD
4,USGS,01011500,"St. Francis River near Connors, New Brunswick",ST,47.206979,-68.956428,168830,00060,2023-05-25T12:30:00Z,EDT,659.0,None,3.0,P0_10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8778,USGS,50129254,"Rio Loco At Las Latas Nr La Joya Nr Guanica, PR",ST,18.007182,-66.876320,124256,00060,2023-05-25T12:00:00Z,AST,0.0,None,0.0,NR_0FLOW,NA
8779,USGS,50136400,"Rio Rosario Nr Hormigueros, PR",ST,18.158099,-67.085388,124261,00060,2023-05-25T11:30:00Z,AST,14.0,None,0.0,P0_10,NOFLOOD
8780,USGS,50138000,"Rio Guanajibo Nr Hormigueros, PR",ST,18.140694,-67.148374,124264,00060,2023-05-25T11:30:00Z,AST,33.8,None,0.0,P25_75,NOFLOOD
8781,USGS,50144000,"Rio Grande De Anasco Nr San Sebastian, PR",ST,18.282174,-67.050584,124274,00060,2023-05-25T12:00:00Z,AST,158.0,None,-10.0,P10_25,NOFLOOD


In [13]:
# dicts containing all the information that pertains to our surface water level data from USGS map
# meaning that this is the network data that is called when https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# when we turn on surface-water levels
# This is all of the surface water level data that exists for USGS - the value that it refers to is gauge height level
# this data will be refreshed everytime we call this code body.

def getSurfaceWaterLevel():
    headers_swl = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_swl = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(SiteTypeCode%20in%20(%27ES%27%2C%27LK%27%2C%27OC%27%2C%27OC-CO%27%2C%27ST%27%2C%27ST-CA%27%2C%27ST-DCH%27%2C%27ST-TS%27%2C%27WE%27))%20and%20(ParameterCode%20in%20(%2700065%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers_swl,
    )
    
    return response_swl.json()['value']

In [14]:
# create a list of all surface water level data for the USGS gauges with the pertinent information
list_swl = getSurfaceWaterLevel()

#Create a dataframe that contains surface water level values
swl_df = pd.DataFrame.from_dict(getSurfaceWaterLevel())

In [15]:
#Rename the Value and Rate of change columns so they do not clash when combining multiple dataframes
swl_df.rename(columns={'Value':'Gauge Height Ft', 'RateOfChangeUnitPerHour':'RateofChange_Gauge'},inplace=True)

In [16]:
swl_df

,AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Gage Height Ft,ValueFlagCode,RateofChange_Gauge,StatisticStatusCode,FloodStageStatusCode
0,USGS,01010000,"St. John River at Ninemile Bridge, Maine",ST,46.700556,-69.715556,65331,00065,2023-05-25T12:00:00Z,EDT,1.96,None,0.00,None,NOFLOOD
1,USGS,01010070,"Big Black River near Depot Mtn, Maine",ST,46.893889,-69.751667,65335,00065,2023-05-25T12:00:00Z,EDT,3.83,None,0.01,None,NOFLOOD
2,USGS,01010500,"St. John River at Dickey, Maine",ST,47.113056,-69.088056,65341,00065,2023-05-25T12:30:00Z,EDT,5.03,None,0.00,None,NOFLOOD
3,USGS,01011000,"Allagash River near Allagash, Maine",ST,47.069722,-69.079444,65347,00065,2023-05-25T12:00:00Z,EDT,3.27,None,0.01,None,NOFLOOD
4,USGS,01011500,"St. Francis River near Connors, New Brunswick",ST,47.206979,-68.956428,168831,00065,2023-05-25T12:30:00Z,EDT,3.66,None,0.01,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,USGS,50136400,"Rio Rosario Nr Hormigueros, PR",ST,18.158099,-67.085388,124262,00065,2023-05-25T11:30:00Z,AST,1.90,None,0.00,P10_25,NOFLOOD
11078,USGS,50138000,"Rio Guanajibo Nr Hormigueros, PR",ST,18.140694,-67.148374,124265,00065,2023-05-25T11:30:00Z,AST,9.92,None,0.00,P25_75,NOFLOOD
11079,USGS,50144000,"Rio Grande De Anasco Nr San Sebastian, PR",ST,18.282174,-67.050584,124275,00065,2023-05-25T12:00:00Z,AST,2.41,None,-0.04,P25_75,NOFLOOD
11080,USGS,50147800,"Rio Culebrinas At Hwy 404 Nr Moca, PR",ST,18.359743,-67.092169,124277,00065,2023-05-25T12:00:00Z,AST,9.26,None,-0.01,P10_25,NOFLOOD


In [17]:
# dicts containing all the information that pertains to our ground water level data from USGS map
# meaning that this is the network data that is called when https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# when we turn on ground-water levels 
# the value in question for these is depth to water - feet below land surface

def getGroundWaterLevel():
    headers_gwl = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        # 'Cookie': '_hjSessionUser_606685=eyJpZCI6Ijg0MTMwZDAyLWJjZTYtNTYwMC1hNzhlLTUxNDA2ZGI0YWY1NyIsImNyZWF0ZWQiOjE2ODM3MzMyNTg5MzUsImV4aXN0aW5nIjp0cnVlfQ==; _ga=GA1.3.2126571673.1683733259; _ga=GA1.4.2126571673.1683733259; __utmc=191289900; _gid=GA1.2.1982434073.1684175396; _gid=GA1.3.1982434073.1684175396; _ga_Q3BG15MFSR=GS1.1.1684253524.4.0.1684253528.0.0.0; _ga_BTG7QB0V1W=GS1.1.1684338231.2.0.1684338231.0.0.0; _gid=GA1.4.1982434073.1684175396; __utma=191289900.2126571673.1683733259.1684427918.1684429964.7; __utmz=191289900.1684429964.7.5.utmcsr=dashboard.waterdata.usgs.gov|utmccn=(referral)|utmcmd=referral|utmcct=/; _ga_232G9MDQL1=GS1.1.1684507551.13.0.1684507553.0.0.0; _ga=GA1.2.2126571673.1683733259; _gat_UA-179298358-1=1',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_gwl = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(SiteTypeCode%20in%20(%27GW%27%2C%27GW-CR%27%2C%27GW-EX%27%2C%27GW-HZ%27%2C%27GW-IW%27%2C%27GW-MW%27%2C%27GW-TH%27%2C%27SB%27%2C%27SB-CV%27%2C%27SB-GWD%27%2C%27SB-TSM%27%2C%27SB-UZ%27))%20and%20(ParameterCode%20in%20(%2730207%27%2C%2730210%27%2C%2730211%27%2C%2730212%27%2C%2730213%27%2C%2762600%27%2C%2762601%27%2C%2762610%27%2C%2762611%27%2C%2762612%27%2C%2762613%27%2C%2762614%27%2C%2762615%27%2C%2762616%27%2C%2762617%27%2C%2762618%27%2C%2762619%27%2C%2762620%27%2C%2762621%27%2C%2762622%27%2C%2762623%27%2C%2762624%27%2C%2763158%27%2C%2763159%27%2C%2763160%27%2C%2763161%27%2C%2772019%27%2C%2772020%27%2C%2772150%27%2C%2772170%27%2C%2772171%27%2C%2772178%27%2C%2772199%27%2C%2772214%27%2C%2772215%27%2C%2772226%27%2C%2772227%27%2C%2772228%27%2C%2772229%27%2C%2772230%27%2C%2772231%27%2C%2772232%27%2C%2772251%27%2C%2772264%27%2C%2772265%27%2C%2772275%27%2C%2772276%27%2C%2772279%27%2C%2772292%27%2C%2772293%27%2C%2772335%27%2C%2772336%27%2C%2799019%27%2C%2799020%27%2C%2799065%27%2C%2799227%27%2C%2700062%27%2C%2700065%27%2C%2700072%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers_gwl,
    )
    
    return response_gwl.json()['value']

In [18]:
# Create a list of the pertinent information for each of the USGS stations that monitors ground water level

list_gwl = getGroundWaterLevel()

gwl_df = pd.DataFrame.from_dict(getGroundWaterLevel())

In [19]:
gwl_df.rename(columns={'Value':'Depth to Water Below Land Surface', 'RateOfChangeUnitPerHour':'RateOfChange_groundwaterlevel'}, inplace=True)

In [20]:
# dicts containing all the information that pertains to our spring water level data from USGS map
# meaning that this is the network data that is called when https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# when we turn on spring water levels 
# the value in question for these is gauge height 

def getSpringWaterLevel():
    headers_spring = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_spring = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(SiteTypeCode%20in%20(%27SP%27))%20and%20(ParameterCode%20in%20(%2730207%27%2C%2730210%27%2C%2730211%27%2C%2730212%27%2C%2730213%27%2C%2762600%27%2C%2762601%27%2C%2762610%27%2C%2762611%27%2C%2762612%27%2C%2762613%27%2C%2762614%27%2C%2762615%27%2C%2762616%27%2C%2762617%27%2C%2762618%27%2C%2762619%27%2C%2762620%27%2C%2762621%27%2C%2762622%27%2C%2762623%27%2C%2762624%27%2C%2763158%27%2C%2763159%27%2C%2763160%27%2C%2763161%27%2C%2772019%27%2C%2772020%27%2C%2772150%27%2C%2772170%27%2C%2772171%27%2C%2772178%27%2C%2772199%27%2C%2772214%27%2C%2772215%27%2C%2772226%27%2C%2772227%27%2C%2772228%27%2C%2772229%27%2C%2772230%27%2C%2772231%27%2C%2772232%27%2C%2772251%27%2C%2772264%27%2C%2772265%27%2C%2772275%27%2C%2772276%27%2C%2772279%27%2C%2772292%27%2C%2772293%27%2C%2772335%27%2C%2772336%27%2C%2799019%27%2C%2799020%27%2C%2799065%27%2C%2799227%27%2C%2700062%27%2C%2700065%27%2C%2700072%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers_spring,
    )
    
    return response_spring.json()['value']

In [21]:
# This is a list of all pertinent spring water level values that we have scraped
list_spring = getSpringWaterLevel()

#Create a dataframe that contains the spring water level data in a more readable form
spring_df = pd.DataFrame.from_dict(getSpringWaterLevel())


In [22]:
spring_df.rename(columns={'Value':'Gauge Height Ft (spring)', 'RateOfChangeUnitPerHour':'RateOfChange_springwaterlevel'}, inplace=True)

In [23]:
# dicts containing all the information that pertains to our percipitation data from USGS map
# meaning that this is the network data that is called when https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# when we turn on precipitation toggle 
# the value in question for these gauges is percipitation - total inches

def getPrecipitation():
    headers_precip = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive',
        # 'Cookie': '_hjSessionUser_606685=eyJpZCI6Ijg0MTMwZDAyLWJjZTYtNTYwMC1hNzhlLTUxNDA2ZGI0YWY1NyIsImNyZWF0ZWQiOjE2ODM3MzMyNTg5MzUsImV4aXN0aW5nIjp0cnVlfQ==; _ga=GA1.3.2126571673.1683733259; _ga=GA1.4.2126571673.1683733259; __utmc=191289900; _gid=GA1.2.1982434073.1684175396; _ga_Q3BG15MFSR=GS1.1.1684253524.4.0.1684253528.0.0.0; _ga_BTG7QB0V1W=GS1.1.1684338231.2.0.1684338231.0.0.0; _gid=GA1.4.1982434073.1684175396; __utma=191289900.2126571673.1683733259.1684427918.1684429964.7; __utmz=191289900.1684429964.7.5.utmcsr=dashboard.waterdata.usgs.gov|utmccn=(referral)|utmcmd=referral|utmcct=/; _ga_232G9MDQL1=GS1.1.1684507551.13.0.1684507553.0.0.0; _ga=GA1.2.2126571673.1683733259; _gat_UA-179298358-1=1',
        'Referer': 'https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response_precipitation = requests.get(
        'https://dashboard.waterdata.usgs.gov/service/cwis/1.0/odata/CurrentConditions?$top=15000&$filter=(AccessLevelCode%20eq%20%27P%27)%20and%20(1%20eq%201%20and%20true)%20and%20(ParameterCode%20in%20(%2772192%27%2C%2772194%27%2C%2799772%27%2C%2700045%27%2C%2700193%27))&$select=AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Value,ValueFlagCode,RateOfChangeUnitPerHour,StatisticStatusCode,FloodStageStatusCode&$orderby=SiteNumber,AgencyCode,ParameterCode,TimeLocal%20desc&caller=National%20Water%20Dashboard%20default',
        headers=headers_precip,
    )
    
    return response_precipitation.json()['value']

In [24]:
# This is a list of all pertinent percipitation values that we have scraped
list_precipitation = getPrecipitation()

#Create a dataframe that contains the precipitation data

precip_df = pd.DataFrame.from_dict(getPrecipitation())

In [25]:
precip_df.rename(columns={'Value':'Precipitation Total Inches', 'RateOfChangeUnitPerHour':'RateOfChangeUnitPerHour_precip'}, inplace=True)

In [26]:
# In these lists we are trying to find if there is location overlap between gauges or if they are each responsible for 
# doing their own specific reading type. We are using site number because same site number suggests that it is the same
# gauge

loc_gwl = [] #groundwater level
loc_swl = [] #surface water level
loc_streamflow = [] #streamflow
loc_spring = [] #spring water levels
loc_precipitation = [] #loc_percipitation

for i in range(len(list_gwl)):
    loc_gwl.append(list_gwl[i]['SiteNumber'])
        
for i in range(len(list_swl)):
    loc_swl.append(list_swl[i]['SiteNumber'])
    
for i in range(len(list_streamflow)):
    loc_streamflow.append(list_streamflow[i]['SiteNumber'])   
    
for i in range(len(list_spring)):
    loc_spring.append(list_spring[i]['SiteNumber']) 
    
for i in range(len(list_precipitation)):
    loc_precipitation.append(list_precipitation[i]['SiteNumber'])  

In [27]:
# We see that there is no overlap between all our variables in streamflow, surface water level, ground water level, spring water level gauges based on their sets

# Stream flow checks
print(set(loc_streamflow) & set(loc_spring))
print(set(loc_streamflow) & set(loc_gwl))
print('New Set\n\n', set(loc_streamflow) & set(loc_swl))
#print('New Set\n\n', set(loc_streamflow) & set(loc_precipitation))

# Spring checks
print('New Set\n\n', set(loc_spring) & set(loc_gwl))
#print('New Set\n\n',set(loc_spring) & set(loc_precipitation))
print('New Set\n\n',set(loc_spring) & set(loc_swl))

# Ground water level checks
print('New Set\n\n',set(loc_gwl) & set(loc_swl))
#print('New Set\n\n',set(loc_gwl) & set(loc_precipitation))

# Final checks (spring water level and precipitation)
#print('New Set\n\n',set(loc_swl) & set(loc_precipitation))




set()
set()
New Set

 {'50025155', '04114498', '01636690', '50111320', '01104415', '09523800', '09073300', '07263296', '06800000', '03419530', '0214183365', '11337080', '01483200', '05458000', '02433496', '02423800', '04087119', '09368000', '05545750', '06889000', '06772100', '02298527', '01157000', '06345780', '02353000', '08158922', '07312610', '06918740', '06893830', '02146110', '13161930', '033483827', '12058790', '02428400', '16274100', '10172800', '08315480', '07052000', '08206910', '02374250', '01105870', '03091500', '04197152', '02162035', '094196783', '03215410', '08017250', '14046000', '04144032', '06412810', '02398000', '02423130', '04126970', '02301740', '07074500', '05536340', '10343000', '03342000', '13011820', '02135200', '01669000', '08045850', '12148000', '04058100', '07365500', '50051800', '05536162', '02300210', '02085000', '03179000', '04031000', '13306370', '03102500', '11504115', '50050900', '13112000', '06713000', '06719505', '02429540', '04247055', '01197145', '

In [28]:
# Based on the results of the sets, we see that 
# swl - stream flow
# stream flow - precipitation 
# spring - precipitation
# ground water = precipitation
# swl - precipitation
# All have overlapping gauges - seems like precipitation is a common addition for these gauges to have

# We want to know if the overlapping gauges are supplying the same 'value' information or if they are supplying 
# information specific to the map information they are displaying - so we will check a few keys from the dicitonary


for i in range(len(list_streamflow)):
    if list_streamflow[i]['SiteNumber'] == '08210000':
        print(list_streamflow[i])
        
for i in range(len(list_swl)):
    if list_swl[i]['SiteNumber'] == '08210000':
        print(list_swl[i])
        
# Based on some very simple experimentation it seems that the values and rate of change are for the specific gauge metrics
# and not static depending on which

{'AgencyCode': 'USGS', 'SiteNumber': '08210000', 'SiteName': 'Nueces Rv nr Three Rivers, TX', 'SiteTypeCode': 'ST', 'Latitude': 28.42749545, 'Longitude': -98.1780625, 'CurrentConditionID': 142023, 'ParameterCode': '00060', 'TimeLocal': '2023-05-25T11:15:00Z', 'TimeZoneCode': 'CDT', 'Value': 609.0, 'ValueFlagCode': None, 'RateOfChangeUnitPerHour': 1.0, 'StatisticStatusCode': 'P25_75', 'FloodStageStatusCode': 'NOFLOOD'}
{'AgencyCode': 'USGS', 'SiteNumber': '08210000', 'SiteName': 'Nueces Rv nr Three Rivers, TX', 'SiteTypeCode': 'ST', 'Latitude': 28.42749545, 'Longitude': -98.1780625, 'CurrentConditionID': 142024, 'ParameterCode': '00065', 'TimeLocal': '2023-05-25T11:15:00Z', 'TimeZoneCode': 'CDT', 'Value': 6.35, 'ValueFlagCode': None, 'RateOfChangeUnitPerHour': 0.01, 'StatisticStatusCode': 'P75_90', 'FloodStageStatusCode': 'NOFLOOD'}


In [29]:
# This is where we will draw our flood warning, flood advisory severe thunderstorm warning from 
# dicts containing all the information that pertains to our atmospheric data from USGS map
# meaning that this is the network data that is called when https://dashboard.waterdata.usgs.gov/app/nwd/en/?region=lower48&aoi=default
# Atmospheric toggle is turned on

def getAtmospheric():
    headers_atmos = {
        'authority': 'idpgis.ncep.noaa.gov',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'origin': 'https://dashboard.waterdata.usgs.gov',
        'referer': 'https://dashboard.waterdata.usgs.gov/',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }

    params = {
        'f': 'json',
        'geometry': '{"spatialReference":{"wkid":4326},"xmax":-16.4,"xmin":-176.4,"ymax":61.5,"ymin":1.5}',
        'geometryPrecision': '4',
        'geometryType': 'esriGeometryEnvelope',
        'inSr': '4326',
        'maxAllowableOffset': '0.03',
        'noCache': '1684522516673',
        'outFields': 'prod_type,issuance,expiration,url',
        'outSr': '4326',
        'returnGeometry': 'true',
        'spatialRel': 'esriSpatialRelIntersects',
        'where': '1=1',
    }

    response_atmos = requests.get(
        'https://idpgis.ncep.noaa.gov/arcgis/rest/services/NWS_Forecasts_Guidance_Warnings/watch_warn_adv/MapServer/1/query',
        params=params,
        headers=headers_atmos,
    )
    
    return response_atmos.json()['features']

In [30]:
#Create a dataframe with the atmospheric data - geometry of floodplains and such
atmos_df = pd.DataFrame.from_dict(getAtmospheric())

In [31]:
#Keep only the prod_type that are flood, severe thunder storm


#Find all of the unique weather types that come from our atmos_df set
weathertype = []
for i in range(len(atmos_df)):
    weathertype.append(atmos_df['attributes'][i].get("prod_type"))
set(weathertype)

#We see that Flash Flood watch, Flood warning, Flood advisory, Flood watch, Sever thunderstorm warning, gale warning, and 
#severe thunderstorm watch are all potentially relevant to our inland flooding model so we keep these for our dataframe

#Make a list that contains the variables we'll keep 
weather_event_keep = ['Flash Flood Watch','Flood Advisory',  'Flood Warning', 'Flood Watch',
 'Gale Warning', 'Severe Thunderstorm Warning', 'Severe Thunderstorm Watch',]

#This drops all of the rows we have deemed non-relevant to our flood estimation
for i in range(len(atmos_df)):
    if atmos_df['attributes'][i].get("prod_type") not in weather_event_keep:
        atmos_df.drop(i, inplace=True)

In [32]:
#Save atmos_df to csv
atmos_df.to_csv('flood_and_severe_weather_geometry.csv')

In [43]:
streamflow_df[(streamflow_df['FloodStageStatusCode'] == '>MAJ')]
swl_df[(swl_df['FloodStageStatusCode'] == '>MAJ')]


,AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Gage Height Ft,ValueFlagCode,RateofChange_Gauge,StatisticStatusCode,FloodStageStatusCode
1982,USGS,02231291,"Nassau River Near Tisonia, FL",ST,30.550793,-81.589818,30482,00065,2023-05-25T12:30:00Z,EDT,12.92,None,0.70,P100,>MAJ
2039,USGS,02244040,"St. Johns R At Buffalo Bluff Near Satsuma, FL",ST,29.596358,-81.683138,30757,00065,2023-05-25T12:15:00Z,EDT,12.03,None,-0.20,P75_90,>MAJ
2041,USGS,02244440,"Dunns Creek Near Satsuma, FL",ST,29.577748,-81.626191,30769,00065,2023-05-25T12:00:00Z,EDT,12.23,None,-0.20,P90_100,>MAJ
2052,USGS,02246459,Cedar River At San Juan Avenue At Jacksonville...,ST,30.282184,-81.740376,30809,00065,2023-05-25T12:00:00Z,EDT,12.32,None,0.10,P100,>MAJ
2053,USGS,02246500,"St. Johns River At Jacksonville, FL",ST,30.322462,-81.665374,30817,00065,2023-05-25T12:00:00Z,EDT,11.80,None,0.30,P100,>MAJ
2054,USGS,02246515,"Pottsburg Creek Nr South Jacksonville, Fla.",ST,30.264407,-81.590092,30824,00065,2023-05-25T11:30:00Z,EDT,20.21,None,-0.02,P25_75,>MAJ
2787,USGS,02444160,"Tombigbee River At Bevill L&D Nr Pickensville, AL",ST,33.210679,-88.288645,166554,00065,2023-05-25T11:00:00Z,CDT,136.32,None,0.01,P10_25,>MAJ
2823,USGS,02462500,Black Warrior R At Bankhead L&D Near Bessemer Al,ST,33.458446,-87.354168,3074,00065,2023-05-25T10:30:00Z,CDT,254.61,None,-0.10,P25_75,>MAJ
4714,USGS,05056500,"Devils Lake Nr Devils Lake, ND",LK,48.099445,-98.903177,92419,00065,2023-05-25T11:00:00Z,CDT,50.69,None,0.01,None,>MAJ
4715,USGS,05056665,"Eastern Stump Lake Nr Lakota, ND",LK,47.868609,-98.357879,92422,00065,2023-05-25T11:15:00Z,CDT,50.51,None,0.01,P25_75,>MAJ


In [ ]:
#Make some changes to our dataframe columns before we move onto merging
#What we are doing is changing the lat/long/sitename parameters so arbitrary parameters are not passed
#And dropping all of the columns that will be repeated between merges and given arbitrary titles

streamflow_df = streamflow_df.drop(columns=['CurrentConditionID', 'ParameterCode', 'TimeLocal', 'TimeZoneCode', 'ValueFlagCode', 'StatisticStatusCode', 'FloodStageStatusCode','SiteTypeCode', 'AgencyCode'])
precip_df = precip_df.drop(columns=['CurrentConditionID', 'ParameterCode', 'TimeLocal', 'TimeZoneCode', 'ValueFlagCode', 'StatisticStatusCode', 'FloodStageStatusCode','SiteTypeCode', 'AgencyCode'])
spring_df.rename(columns={'SiteName': 'SiteName_springwaterlevel', 'Latitude':'Latitude_springwaterlevel', 'Longitude':'Longitude_springwaterlevel'})
spring_df = spring_df.drop(columns=['CurrentConditionID', 'ParameterCode', 'TimeLocal', 'TimeZoneCode', 'ValueFlagCode', 'StatisticStatusCode', 'FloodStageStatusCode','SiteTypeCode', 'AgencyCode'])
gwl_df = gwl_df.drop(columns=['CurrentConditionID', 'ParameterCode', 'TimeLocal', 'TimeZoneCode', 'ValueFlagCode', 'StatisticStatusCode', 'FloodStageStatusCode','SiteTypeCode', 'AgencyCode'])
swl_df = swl_df.drop(columns=['CurrentConditionID', 'ParameterCode', 'TimeLocal', 'TimeZoneCode', 'ValueFlagCode', 'StatisticStatusCode', 'FloodStageStatusCode','SiteTypeCode', 'AgencyCode'])

In [ ]:
streamflow_df.rename(columns={'SiteName': 'SiteName_streamflow', 'Latitude':'Latitude_streamflow', 'Longitude':'Longitude_streamflow'}, inplace=True)
spring_df.rename(columns={'SiteName': 'SiteName_springwaterlevel', 'Latitude':'Latitude_springwaterlevel', 'Longitude':'Longitude_springwaterlevel'},inplace=True)
gwl_df.rename(columns={'SiteName': 'SiteName_groundwaterlevel', 'Latitude':'Latitude_groundwaterlevel', 'Longitude':'Longitude_groundwaterlevel'},inplace=True)
swl_df.rename(columns={'SiteName': 'SiteName_surfacewaterlevel', 'Latitude':'Latitude_surfacewaterlevel', 'Longitude':'Longitude_surfacewaterlevel'},inplace=True)
precip_df.rename(columns={'SiteName': 'SiteName_precip', 'Latitude':'Latitude_precip', 'Longitude':'Longitude_precip'},inplace=True)


In [ ]:
#Get all the different column values together 
list_streamflow_columns = streamflow_df.columns.values
list_spring_columns = spring_df.columns.values
list_gwl_columns = gwl_df.columns.values
list_swl_columns = swl_df.columns.values
list_precip_columns = precip_df.columns.values

#Create a combined list of all the column names that we will have to use
columns_all = itertools.chain(list_gwl_columns, list_precip_columns, list_swl_columns,list_spring_columns, list_streamflow_columns)
columns_all = list(set(columns_all))

In [ ]:
#Create a dataframe with the new columns
#We will come back to this for our second run our model as we will need all of our variables to determine levels
df = pd.DataFrame(columns=columns_all)

In [47]:
#import our elevation data into a usable dataset
elevation_data = pd.read_csv('Structure_Elevation_Test.csv')

In [53]:
#We want to find ~5 nearest points of elevation for our gauges - which means we will
#use some lat,long libraries that can quickly calculate that distance
import geopy.distance

# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)

# print(geopy.distance.geodesic(coords_1, coords_2).miles)

173.5818455248231


In [167]:
swl_df_LA = swl_df[swl_df['SiteNumber'] == '08030500']

In [170]:
swl_df_LA = swl_df_LA.drop(columns=['nearest_structure(ft)', 'estimated_elevation'])
swl_df_LA

,AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Gage Height Ft,ValueFlagCode,RateofChange_Gauge,StatisticStatusCode,FloodStageStatusCode
7467,USGS,08030500,"Sabine Rv nr Ruliff, TX",ST,30.303817,-93.743778,139609,00065,2023-05-25T10:45:00Z,CDT,24.49,None,-0.01,P75_90,>MNR


In [171]:
#This is just a test for Louisiana
#swl_df_LA = swl_df[swl_df['SiteName'].str.contains('LA')].reset_index()

#Add a column to surface water level that contains the estimated surface water level elevation
swl_df_LA['nearest_structure(ft)'] = 0.00
swl_df_LA['estimated_elevation'] = 0.00
swl_df_LA

,AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Gage Height Ft,ValueFlagCode,RateofChange_Gauge,StatisticStatusCode,FloodStageStatusCode,nearest_structure(ft),estimated_elevation
7467,USGS,08030500,"Sabine Rv nr Ruliff, TX",ST,30.303817,-93.743778,139609,00065,2023-05-25T10:45:00Z,CDT,24.49,None,-0.01,P75_90,>MNR,0.0,0.0


In [175]:
x = swl_df_LA['Latitude']
y = swl_df_LA['Longitude']

In [176]:
elevation_data

,OID_,Join_Count,TARGET_FID,latitude,longitude,ContourEle
0,1,1,1,30.533078,-92.688051,35
1,2,1,2,30.818390,-92.647130,115
2,3,1,3,30.808949,-92.661303,110
3,4,1,4,30.820524,-92.659002,115
4,5,1,5,30.843639,-92.689672,105
...,...,...,...,...,...,...
7711,7712,1,7712,30.889211,-92.918131,130
7712,7713,1,7713,30.889090,-92.919797,140
7713,7714,1,7714,30.887433,-92.920090,140
7714,7715,1,7715,30.885541,-92.925450,150


In [174]:
for i in range(len(elevation_data))

7467    35.303817
Name: Latitude, dtype: float64

In [118]:
swl_df_LA['Latitude'].reset

7467    30.303817
Name: Latitude, dtype: float64

In [87]:
elevation_data

,OID_,Join_Count,TARGET_FID,latitude,longitude,ContourEle
0,1,1,1,30.533078,-92.688051,35
1,2,1,2,30.818390,-92.647130,115
2,3,1,3,30.808949,-92.661303,110
3,4,1,4,30.820524,-92.659002,115
4,5,1,5,30.843639,-92.689672,105
...,...,...,...,...,...,...
7711,7712,1,7712,30.889211,-92.918131,130
7712,7713,1,7713,30.889090,-92.919797,140
7713,7714,1,7714,30.887433,-92.920090,140
7714,7715,1,7715,30.885541,-92.925450,150


In [177]:
swl_df[swl_df['SiteNumber'] == '01109403']

,AgencyCode,SiteNumber,SiteName,SiteTypeCode,Latitude,Longitude,CurrentConditionID,ParameterCode,TimeLocal,TimeZoneCode,Gage Height Ft,ValueFlagCode,RateofChange_Gauge,StatisticStatusCode,FloodStageStatusCode
188,USGS,01109403,"Ten Mile R., Pawtucket Ave. At E. Providence, RI",ST,41.830934,-71.350331,66291,00065,2023-05-25T12:15:00Z,EDT,3.32,None,-0.01,None,None
